# 모폴로지(morphology)
- 모폴로지는 **형태학**이란 뜻으로서 다양한 학문 분야에서 사용되는 용어이다 
- 영상처리에서는 모폴로지는 영상의 객체들의 형태(shape)를 분석하고 처리하는 기법이다. 이 형태학적 처리를 활용하여 영상의 경계, 골격, 블록 등의 형태를 표현하는데 필요한 요소를 추출한다. 간단히 말하면, `영상 내에 존재하는 객체의 형태를 조금식 변형시킴으로써 영상 내에서 불필요한 잡음을 제거하거나 객체를 두력하게 하여 필요한 요소를 추출한다`

### 침식 연산 (erosion operation)
- 침식연산은 객체를 침식시키는 연산이다. 다라서 객체의 크기가 축소되고, 배경은 확장된다 
- 객체의 크기가 축소되기 대문에 영상 내에 존재하는 잡음 같은 작은 크기의 객체들은 사라질 수 있다 
- 이러한 연산을 이용하여 소금-후추(salt & papper) 잡음과 같은 임펄스 (impulse) 잡음 들을 제거할 수 있다. 또한 영상 내에서 객체의 돌출부를 감소키기기 대문에 서로 닿은 물체를 분리할 때도 유용하게 사용할 수 잇다 

![erosion](img/erosion.png)

- 입력 영상의 화소와 마스크 원소가 한개의 화소라도 일치하지 않으면 출력 화소는 검은색 0이 된다. 반면, 입력 영상의 화소와 마스크 원소와 비교해서 모두가 일치하면 출력 화소는 흰색인 1이 지정된다. 여기서 마스크 원소가 1인 워치만을 비교대상으로 한다. 도한 마스크의 크기와 원소의 구성은 입력 영상의 형태에 다라서 사용자가 조정하여 더 나은 결과 영상을 생성할 수 있다.

In [1]:
import numpy as np, cv2

In [2]:
def erode(img, mask=None):
    dst = np.zeros(img.shape, np.uint8)
    if mask is None: mask = np.ones((3, 3), np.uint8)
    ycenter, xcenter = np.divmod(mask.shape[:2], 2)[0]

    mcnt = cv2.countNonZero(mask)
    for i in range(ycenter, img.shape[0] - ycenter):           # 입력 행렬 반복 순회
        for j in range(xcenter, img.shape[1] - xcenter):
            y1, y2 = i - ycenter, i + ycenter + 1              # 마스크 높이 범위
            x1, x2 = j - xcenter, j + xcenter + 1              # 마스크 너비 범위
            roi = img[y1:y2, x1:x2]                            # 마스크 영역
            temp = cv2.bitwise_and(roi, mask)
            cnt  =  cv2.countNonZero(temp)                     # 일치한 화소수 계산
            dst[i, j] = 255 if (cnt == mcnt) else 0            # 출력 화소에 저장
    return dst


In [3]:
image = cv2.imread("img/morph.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상파일 읽기 오류")

In [4]:
data = [0, 1, 0,                                               # 마스크 선언 및 초기화
        1, 1, 1,
        0, 1, 0]
mask = np.array(data, np.uint8).reshape(3, 3)
th_img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)[1]  # 영상 이진화

In [5]:
dst1 = erode(th_img, mask)                                     # 사용자 정의 침식 함수
dst2 = cv2.erode(th_img, mask)
# dst2 = cv2.morphologyEx(th_img, cv2.MORPH_ERODE, mask)         # OpenCV의 침식 함수

In [6]:
cv2.imshow("image", image)
cv2.imshow("binary image", th_img)
cv2.imshow("User erode", dst1)
cv2.imshow("OpenCV erode", dst2)
cv2.waitKey(0)

-1

![result](img/13_result.png)

- binary imgae은 왼쪽의 입력 영상을 이진 영상으로 만들어 모폴로지 처리를 준비한 영상이다.
- User erode는 침식 연산을 구현해서 수행한 결과 
- OpenCV erode는 opencv에서 제공하는 함수를 사용하여 수행한 결과
- 연산의 결과로 배경 부분에 있는 잡음들(흰색)이 제거 된것을 볼수 있다. 그런데 객체가 침식되므로 객체(글자)의 내부의 작은 공간(검은색)은 더 넓어진다. 